# CLIPIT PixelDraw with palette enforcement


This version has two new functions: palette enforcement and smoothness enforcement.

Code:- https://github.com/altsoph/clipit

In [ ]:
#@markdown # **Setup**


#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left. For setup,
#@markdown **you need to run this cell,
#@markdown then choose Runtime -> Restart Runtime from the menu,
#@markdown and then run the cell again**. It should remind you to
#@markdown do this after the first run.

#@markdown Setup can take 5-10 minutes, but once it is complete it usually does not need to be repeated
#@markdown until you close the window.

#@markdown **Note**: This installs the software on the Colab 
#@markdown notebook in the cloud and not on your computer.

# https://stackoverflow.com/a/56727659/1010653

# Add a gpu check
# (this can get better over time)
from google.colab import output

nvidia_output = !nvidia-smi --query-gpu=memory.total --format=noheader,nounits,csv
gpu_memory = int(nvidia_output[0])
if gpu_memory < 14000:
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
  warning_string = f"--> GPU check: ONLY {gpu_memory} MiB available: WARNING, THIS IS PROBABLY NOT ENOUGH <--"
  print(warning_string)
  output.eval_js('alert("Warning - low GPU (see message)")')
else:
  print(f"GPU check: {gpu_memory} MiB available: this should be fine")

from IPython.utils import io
with io.capture_output() as captured:
  !git clone https://github.com/openai/CLIP
  # !pip install taming-transformers
  !git clone https://github.com/CompVis/taming-transformers.git
  !rm -Rf clipit
  !git clone -b enforce_palette_annealing --single-branch git://github.com/altsoph/clipit.git
  !pip install ftfy regex tqdm omegaconf pytorch-lightning
  !pip install kornia
  !pip install imageio-ffmpeg   
  !pip install einops
  !pip install torch-optimizer
  !pip install easydict
  !pip install braceexpand
  !pip install git+https://github.com/pvigier/perlin-numpy

  # ClipDraw deps
  !pip install svgwrite
  !pip install svgpathtools
  !pip install cssutils
  !pip install numba
  !pip install torch-tools
  !pip install visdom

  !git clone https://github.com/BachiLi/diffvg
  %cd diffvg
  # !ls
  !git submodule update --init --recursive
  !python setup.py install
  %cd ..

import sys
sys.path.append("clipit")

result_msg = "setup complete"
import IPython
import os
if not os.path.isfile("first_init_complete"):
  # put stuff in here that should only happen once
  !mkdir -p models
  os.mknod("first_init_complete")
  result_msg = "Please choose Runtime -> Restart Runtime from the menu, and then run Setup again"

js_code = f'''
document.querySelector("#output-area").appendChild(document.createTextNode("{result_msg}"));
'''
js_code += '''
for (rule of document.styleSheets[0].cssRules){
  if (rule.selectorText=='body') break
}
rule.style.fontSize = '30px'
'''
display(IPython.display.Javascript(js_code))
#@markdown ---

In [ ]:
#@markdown #**Settings**

#@markdown Enter a description of what you want to draw - I usually add #pixelart to the prompt.
#@markdown If PixelDraw is not used, it will use VQGAN instead.
#@markdown <br>

prompts = "MF Doom #pixelart" #@param {type:"string"}
output_path = "/content/output

init_image = "" #@param {type:"string"}

aspect = "widescreen" #@param ["widescreen", "square"]

use_pixeldraw = True #@param {type:"boolean"}

#@markdown Specify the desired palette ("" for default), here are examples of what can be parsed:
#@markdown * white   (16 color black to white ramp)
#@markdown * red     (16 color black to red ramp)
#@markdown * rust\8  (8 color black to rust ramp)
#@markdown * red->rust         (16 color red to rust ramp)
#@markdown * red->#ff0000      (16 color red to yellow ramp)
#@markdown * red->#ff0000\20   (20 color red to yellow ramp)
#@markdown * black->red->white (16 color black/red/white ramp)
#@markdown * [black, red, #ff0000] (three colors)
#@markdown * [#000000, #ff0000, #ffff00, #000080] (four colors)
#@markdown * red->white;blue->yellow (32 colors across two ramps of 16)
#@markdown * red;blue;yellow         (48 colors from combining 3 ramps)
#@markdown * red\8;blue->yellow\8    (16 colors from combining 2 ramps)
#@markdown * red->yellow;[black]     (16 colors from ramp and also black)
#@markdown * (for example: ):

use_palette = "" #@param {type:"string"}
#@markdown Use this flag to activate the smoothess enforcement:
force_smoothness = True #@param {type:"boolean"} 

#@markdown Use this flag to activate the saturation enforcement (use it against color fading):
force_saturation = True #@param {type:"boolean"} 

#@markdown When you have the settings you want, press the play button on the left.
#@markdown The system will save these and start generating images below.

#@markdown When that is done you can change these
#@markdown settings and see if you get different results. Or if you get
#@markdown impatient, just select "Runtime -> Interrupt Execution".
#@markdown Note that the first time you run it may take a bit longer
#@markdown as nessary files are downloaded.


#@markdown
#@markdown *Advanced: you can also edit this cell and add add additional
#@markdown settings, combining settings from different notebooks.*


# Simple setup
import clipit

# these are good settings for pixeldraw
clipit.reset_settings()
clipit.add_settings(prompts=prompts, aspect=aspect)
clipit.add_settings(quality="better", scale=2.5)
clipit.add_settings(use_pixeldraw=use_pixeldraw)
clipit.add_settings(iterations=500, display_every=50, save_every=20, make_video=True, output=output_path+"/output.png")

# palette = None
if use_palette and use_palette!='None':
  clipit.add_settings(target_palette=use_palette)
# if force_palette == "custom" and use_palette.strip():
#   import matplotlib.colors
#   palette = [matplotlib.colors.to_rgb(c.strip()) for c in use_palette.strip().split(',')]
# elif force_palette == "green":
#   palette = [[0.44*i*16/256,i*16/256,0.53*i*16/256] for i in range(16)]
# elif force_palette == "orange":
#   palette = [[i*16/256,0.8*i*16/256,0.2*i*16/256] for i in range(16)]
# elif force_palette == "blue":
#   palette = [[0.44*i*16/256,0.53*i*16/256,i*16/256] for i in range(16)]
# elif force_palette == "red":
#   palette = [[i*16/256,0.53*i*16/256,0.44*i*16/256] for i in range(16)]
# elif force_palette == "grayscale":
#   palette = [[i*16/256,i*16/256,i*16/256] for i in range(16)]
# palette = 'red'
# if palette and palette_annealing:
#   clipit.add_settings(target_palette=palette) #, enforce_palette_annealing=palette_annealing)

if force_smoothness and force_smoothness!='None':
  clipit.add_settings(enforce_smoothness=50, enforce_smoothness_type='log')
  # if force_smoothness in ('clipped','log'):
  #   clipit.add_settings(enforce_smoothness_type=force_smoothness)

if force_saturation:
  clipit.add_settings(enforce_saturation=1000)

clipit.add_settings(noise_prompt_seeds=[1,2,3])


#### YOU CAN ADD YOUR OWN CUSTOM SETTING HERE ####
# this is the example of how to run longer with less frequent display
# clipit.add_settings(iterations=500, display_every=50)

settings = clipit.apply_settings()
clipit.do_init(settings)
clipit.do_run(settings)
!sleep 10

#@markdown ---

JS to prevent idle timeout:

Press F12 OR CTRL + SHIFT + I OR right click on this website -> inspect.
Then click on the console tab and paste in the following code.

```javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
```